### **1\. Spread와 구조 분해가 느린 이유: 내부 동작과 성능 비용**

Spread(...) 문법은 객체와 배열을 손쉽게 복사하거나 병합할 수 있어 매우 편리합니다.  
그러나 이 편리함 뒤에는 잘 드러나지 않는 성능 비용이 숨어 있습니다.  

**🔷 Spread 구문(...)의 숨겨진 비용과 성능 영향**  
**1) 전체 순회(Iteration) 발생**  
Spread는 원본 객체의 모든 속성(key) 또는 배열의 모든 요소(element)를 처음부터 끝까지 하나씩 읽어와야 합니다.  
즉, 배경에서는 완전한 순회 작업이 이루어집니다.

**2) 새로운 객체 또는 배열 생성**  
Spread는 단순한 참조 복사가 아니라, 항상 새로운 객체 또는 배열을 생성합니다.  
이 과정에서 속성 복사 비용이 추가로 발생합니다.

**3) 얕은 복사(Shallow Copy)만 수행**  
Spread는 내부 객체까지 깊게 복사하지 않습니다.  
중첩된 객체는 기존 객체의 참조를 그대로 공유합니다. 

**✔️ 예시: Spread의 동작과 얕은 복사 확인하기**


In [ ]:
const user = {
  id: 101,
  name: 'Alice',
  settings: { theme: 'dark' }
};

// ❌ 이 시점에서 user 객체의 모든 키('id', 'name', 'settings')를 순회해서 새 객체를 만듭니다.
const copy = { ...user, isLoggedIn: true };

console.log('원본 user:', user);
console.log('복사본 copy:', copy);

// settings는 얕은 복사이므로 같은 객체를 가리킵니다.
console.log('settings 동일 참조 여부:', copy.settings === user.settings); // true

// 복사본의 settings를 수정해 봅니다.
copy.settings.theme = 'light';

console.log('user.settings.theme:', user.settings.theme);   // 'light'
console.log('copy.settings.theme:', copy.settings.theme);   // 'light'

**🔷 실무 최적화: 복사 대신 참조 유지와 직접 수정**  
성능이 중요하고, 원본 데이터의 직접 수정이 허용되는 상황이라면 불필요한 복사를 만들지 않는 것이 가장 좋은 최적화입니다.  
특히 대량 데이터에 대한 반복 처리에서 매번 Spread로 새 객체를 만드는 패턴은 피하는 것이 좋습니다.  

**✔️ 나쁜 예: Spread 남용으로 인한 성능 저하**  

In [2]:
// 10만 개의 데이터가 담긴 배열을 준비합니다.
const bigList = Array.from({ length: 100_000 }, (_, i) => ({
  id: i,
  name: `item-${i}`,
  status: 'PENDING',
}));

console.time('spread-map');

// 매 순회마다 item 객체를 Spread로 복사하여 새로운 객체를 만듭니다.
const resultWithSpread = bigList.map(item => ({
  ...item,          // 🔥 매번 전체 속성을 복사
  status: 'DONE',
}));

console.timeEnd('spread-map');
console.log(resultWithSpread[0]);

spread-map: 14.9ms
{ id: 0, name: "item-0", status: "DONE" }


**✔️ 좋은 예: 참조 기반 수정으로 복사 비용 제거**

In [3]:
// 같은 bigList를 사용합니다.
const bigList2 = Array.from({ length: 100_000 }, (_, i) => ({
  id: i,
  name: `item-${i}`,
  status: 'PENDING',
}));

console.time('mutate-map');

// 새로운 객체를 만들지 않고 기존 객체의 속성만 수정합니다.
const resultWithMutation = bigList2.map(item => {
  item.status = 'DONE';  // 🔁 여기서만 값 변경
  return item;           // 기존 객체(참조)를 그대로 반환
});

console.timeEnd('mutate-map');
console.log(resultWithSpread[0]);

mutate-map: 2.45ms
{ id: 0, name: "item-0", status: "DONE" }


**🔷 안전한 깊은 복사: structuredClone 사용하기**  
과거에는 JSON.parse(JSON.stringify(obj)) 방식이 많이 사용되었지만, 이 방법에는 다음과 같은 문제가 있습니다.  
▸ Date, Map, Set, RegExp, 함수 등 복잡한 타입이 제대로 복사되지 않음  
▸ 순환 참조(circular reference)가 있을 경우 에러 발생  
▸ 타입 정보 손실로 인해 버그를 유발할 수 있음  
이러한 단점을 보완하기 위해 structuredClone(obj) (Node 17+, ES2022+) 이 표준으로 도입되었습니다.  

**✔️ 예시: JSON 방식과 structuredClone 비교**

In [ ]:
const complexData = {
  id: 1,
  time: new Date(),          // Date 객체
  cache: new Map([['key', 'value']]),  // Map 객체
};

// ❌ JSON 방식: 타입 손상 발생
const jsonClone = JSON.parse(JSON.stringify(complexData));

console.log('JSON Clone - time 타입:', jsonClone.time instanceof Date); // false
console.log('JSON Clone - cache 타입:', jsonClone.cache instanceof Map); // false
console.log('JSON Clone - time 값:', jsonClone.time); // 문자열로 변환됨
console.log('JSON Clone - cache 값:', jsonClone.cache); // 일반 객체 형태

// ✅ structuredClone: 타입을 보존한 깊은 복사
const safeClone = structuredClone(complexData);

console.log('structuredClone - time 타입:', safeClone.time instanceof Date); // true
console.log('structuredClone - cache 타입:', safeClone.cache instanceof Map); // true
console.log('structuredClone - time 값:', safeClone.time);   // Date 객체
console.log('structuredClone - cache.get("key"):', safeClone.cache.get('key')); // 'value'

### **2\. 메모리 누수의 진짜 원인: 클로저(Closure)와 WeakMap**  
자바스크립트는 가비지 컬렉션(GC)이 자동으로 메모리를 관리하지만, 코드가 특정 객체를 계속 참조하고 있는 경우 GC는 그 객체를 해제할 수 없습니다.  
이렇게 사용하지 않는 객체가 계속 메모리에 남아있는 현상을 바로 “메모리 누수(Memory Leak)”라고 합니다.  

**🔷 메모리 누수의 위험성과 클로저(Closure) 관리**  
> “클로저는 함수가 선언될 당시의 스코프(변수 환경)를 기억하고,  
> 그 함수가 실행되는 시점에도 그 환경에 접근할 수 있게 하는 기능”입니다.

**✔️ 문제 코드: 클로저가 대형 데이터를 붙잡는 경우**

In [ ]:
function createCacheHandler() {
  // 100만 개의 숫자를 가진 대형 배열 생성
  const hugeData = new Array(1_000_000).fill(0);

  // 이 함수가 hugeData를 참조 → GC가 hugeData를 해제하지 못함
  return index => hugeData[index];
}

const handler = createCacheHandler();

// hugeData는 handler가 유지되는 동안 메모리에 계속 남습니다.
console.log(handler(0)); // 0

0


**✔️ 클래스 기반으로 재작성 (명확하고 관리 용이)**  
클래스 방식은 상태가 인스턴스 내부에 명확하게 저장되기 때문에 초보 개발자에게 더 직관적이고, 메모리 구조를 이해하기도 쉽습니다.


In [8]:
class DataCache {
  constructor(size) {
    // 큰 데이터가 인스턴스 내부에 명시적으로 존재
    this.hugeData = new Array(size).fill(0);
  }

  get(index) {
    return this.hugeData[index];
  }

  clear() {
    // 명시적으로 메모리 해제 가능
    this.hugeData = null;
  }
}

// 사용 예시
const cache = new DataCache(1_000_000);
console.log(cache.get(0)); // 0

// 더 이상 필요 없으면 메모리 해제
cache.clear();

0


**✔️ 개선 코드: 필요한 값만 클로저에 남기기**  
대규모 데이터가 클로저 내부에 들어가지 않도록 “정말 필요한 최소한의 값만” 클로저 환경에 남기는 것이 가장 안전합니다.

In [9]:
function createCounter() {
  let count = 0;  // 매우 작은 데이터만 클로저에 저장
  return () => ++count;
}

const counter = createCounter();

console.log(counter()); // 1
console.log(counter()); // 2

1
2


**✔️ 개선 코드: 클래스 기반 Counter 구현**

In [10]:
class Counter {
  constructor() {
    // 상태를 인스턴스의 프로퍼티로 명확하게 저장
    this.count = 0;
  }

  next() {
    this.count += 1;
    return this.count;
  }
}
// 사용 예시
const counter = new Counter();
console.log(counter.next()); // 1
console.log(counter.next()); // 2
console.log(counter.next()); // 3

1
2
3


**🔷 WeakMap을 활용한 자동 청소 캐시 패턴**   
WeakMap은 key에 대해 약한 참조(Weak Reference)를 유지합니다.  
→ WeakMap의 key를 다른 곳에서 참조하지 않게 되면, GC는 key와 그에 연결된 value를 자동으로 정리합니다.

**✔️ 예시: WeakMap으로 캐시 만들기**  

In [11]:
// WeakMap을 캐시로 사용합니다.
const computeCache = new WeakMap();

/**
 * 특정 객체를 key로 사용하여 무거운 계산 결과를 캐싱합니다.
 */
function getHeavyResult(inputObj) {
  if (computeCache.has(inputObj)) {
    console.log('캐시 히트!');
    return computeCache.get(inputObj);
  }

  // 2. 실제로는 CPU 사용이 많은 작업이라고 가정합니다.
  const result = inputObj.value * 123456;
  computeCache.set(inputObj, result);

  console.log('새로운 값 계산 완료.');
  return result;
}

// 테스트용 객체 생성
let data = { value: 10 };
// 최초 호출 → 계산 수행
console.log('결과:', getHeavyResult(data)); // 계산 후 결과 출력
// 두 번째 호출 → 캐시 사용
console.log('결과:', getHeavyResult(data)); // 캐시 히트
// 객체를 더 이상 사용하지 않음
data = null;
// 이제 GC가 일어나면 WeakMap 내부의 캐시도 자동으로 제거됩니다.
// (GC는 환경에 따라 즉시 일어나지 않을 수 있습니다.)

새로운 값 계산 완료.
결과: 1234560
캐시 히트!
결과: 1234560


null

### **3\. 반복문 선택이 성능을 좌우하는 이유**  

**🔷 성능에 최적화된 반복문 선택**  
자바스크립트 엔진(V8)의 최적화 관점에서 볼 때, 반복문은 아래 순서로 빠른 경향이 있습니다.

| 형태 | 특징 | 권장 예시 |  
| --- | --- | --- |  
| for (let i = 0; i < len; i++) | 가장 빠른 반복문. 불필요한 함수 호출 없음 | 극한의 성능, 반복/합산 처리 |  
| for...of | 빠름. 내부적으로 이터레이터를 사용하지만 가독성이 우수 | 가장 단순한 순회가 필요할 때 |  
| forEach | 함수 호출 비용 존재 | 가독성이 중요할 때 |  
| map / filter / reduce | 가장 느림. 콜백 호출 + 새로운 배열 생성 비용 발생 | 선언적 코드가 필요할 때 |  

In [18]:
// 20만 개 숫자로 구성된 배열 생성
const numbers = new Array(200_000).fill(1);

// 1) reduce
console.time('reduce');
let sumReduce = numbers.reduce((acc, cur) => acc + cur, 0);
console.timeEnd('reduce');

// 2) forEach
console.time('forEach');
let sumForEach = 0;
numbers.forEach(num => (sumForEach += num));
console.timeEnd('forEach');

// 3) for...of
console.time('forOf');
let sumForOf = 0;
for (const num of numbers) {
  sumForOf += num;
}
console.timeEnd('forOf');

// 4) for (기본형) → 가장 빠른 방식
console.time('for');
let sumFor = 0;
for (let i = 0; i < numbers.length; i++) {
  sumFor += numbers[i];
}
console.timeEnd('for');

reduce: 1.79ms
forEach: 2.63ms
forOf: 3.55ms
for: 1.24ms


**🔷 고차 함수(map, reduce) 사용 시 성능 저하 이유** 

In [19]:
// 10만 개의 숫자를 가진 배열 생성
const numbers = new Array(100_000).fill(1);

// ❌ reduce 사용: 매 요소마다 콜백이 호출되므로 비용이 큼
console.time('Reduce_Time');
const totalA = numbers.reduce((acc, cur) => acc + cur, 0);
console.timeEnd('Reduce_Time');

// ✅ for 루프 사용: 가장 빠른 연산
console.time('For_Time');
let totalB = 0;
for (let i = 0; i < numbers.length; i++) {
  totalB += numbers[i];
}
console.timeEnd('For_Time');

Reduce_Time: 0.965ms
For_Time: 0.954ms


**🔷 GC Pause와 임시 객체 최소화**  

**✔️ 문제 코드: 매번 새로운 대형 배열 생성 → GC 폭증**  

In [20]:
function processData(size) {
  // ❌ 매 호출마다 큰 배열을 새로 생성
  const temp = new Array(size).fill(0);
  return temp[0];
}

console.time('bad');
for (let i = 0; i < 5000; i++) {
  processData(50_000);   // 반복적으로 큰 배열 생성
}
console.timeEnd('bad');

bad: 1090ms


**✔️ 개선 코드: 배열 재사용**  

In [21]:
// 재사용 가능한 버퍼 풀
const bufferPool = {
  buffer: [],
  acquire(size) {
    if (this.buffer.length >= size) return this.buffer;
    this.buffer = new Array(size).fill(0);
    return this.buffer;
  }
};

function processOptimized(size) {
  // ✔ 항상 같은 배열을 재사용
  const temp = bufferPool.acquire(size);
  return temp[0];
}

console.time('good');
for (let i = 0; i < 5000; i++) {
  processOptimized(50_000);
}
console.timeEnd('good');

good: 1.04ms
